<a href="https://colab.research.google.com/github/HuanAII/Sentiment_based_classification_LoRa/blob/main/Sentiment_based_Classification_LoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq --upgrade pip
!pip install -qq --upgrade peft transformers accelerate bitsandbytes datasets trl huggingface_hub evaluate

In [ ]:
#load dataset
from datasets import load_dataset

dataset = load_dataset("uitnlp/vietnamese_students_feedback")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})


In [ ]:
print(dataset['train'][0])

{'sentence': 'slide giáo trình đầy đủ .', 'sentiment': 2, 'topic': 1}


In [ ]:
print(dataset['train'].features['topic'].names)
dataset['train'].features['sentiment'].names


['lecturer', 'training_program', 'facility', 'others']


['negative', 'neutral', 'positive']

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_id = "meta-llama/Llama-2-7b"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
def preprocessing_function(examples):
  inputs = examples['sentence']
  model_inputs = tokenizer(inputs, truncation = True , max_length=512)
  model_inputs['label']= examples["label"]
  return model_inputs

tokenized_dataset = dataset.map(preprocessing_function , batched= True)



#Load base model

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=3, device_map="auto", load_in_4bit=True, use_auth_token=True)


OSError: Can't load tokenizer for 'meta-llama/Llama-2-7b'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-2-7b' is the correct path to a directory containing all relevant files for a LlamaTokenizerFast tokenizer.

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

# Tải mô hình và tokenizer từ mô hình Llama khác
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat")
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat.
401 Client Error. (Request ID: Root=1-680f50b2-79a11dc70b6c55731b2e6144;2ab965fa-0efa-4583-ac7b-b47d0f775839)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
from peft import LoraConfig , get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_steps=100,
    learning_rate=2e-4,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

trainer.train()


In [ ]:
trainer.evaluate()
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")


# **Inference**

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load model và tokenizer từ file
model = AutoModelForSequenceClassification.from_pretrained("./sentiment_model")
tokenizer = AutoTokenizer.from_pretrained("./sentiment_model")

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits

    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    predicted_label = torch.argmax(probabilities, dim=-1).item()

    sentiment = ["Negative", "Neutral", "Positive"]

    return sentiment[predicted_label], probabilities[0][predicted_label].item()

text = "Giảng viên giảng bài rất dễ hiểu."
sentiment, probability = predict_sentiment(text)
print(f"Sentiment: {sentiment}, Probability: {probability:.4f}")
